In [ ]:
# Python libraries
import argparse
import os

import torch
import numpy as np
import sys
sys.path.append("../")
# sys.path.append("/p300/liyuwei/MRI_Bonenet/MedicalZooPytorch")

import lib.medloaders as medical_loaders
import lib.medzoo as medzoo
import lib.train as train
# Lib files
import lib.utils as utils
from lib.losses3D.JoinLoss import JoinLoss
import json
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
seed = 1777777
torch.manual_seed(seed)

In [ ]:
parser = get_arguments()
utils.reproducibility(parser, seed)
# "/p300/liyuwei/MRI_Bonenet/saved_models/MRIBONENET_checkpoints"
args = argparse.ArgumentParser()
args = args.parse_args()
with open(parser, 'r') as f:
    args.__dict__ = json.load(f)

print(args)

os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
print("Number of available GPUs: {}".format(torch.cuda.device_count()))

training_generator, val_generator, full_volume, affine = \
                medical_loaders.generate_datasets(args, path='/p300/liyuwei/DATA_mri/Hand_MRI_capture/seg_final')
model, optimizer = medzoo.create_model(args)
criterion = JoinLoss(classes=args.classes, skip_index_after=args.classes)

if args.cuda:
    model = model.cuda()
    print("Model transferred in GPU.....")

trainer = train.Trainer(args, model, criterion, optimizer, train_data_loader=training_generator,
                        valid_data_loader=val_generator, lr_scheduler=None)
print("START TRAINING...")
trainer.training()
